# Converting the PDF files into vector database  

# 1. Initial setup
## 1.1. Imports
This setup includes loading environment variables from a `.env` file, setting the required environment variables, and importing the necessary modules for further processing. It ensures that the code has access to the required APIs and functions for the subsequent tasks.

In [12]:
# Initial imports
from dotenv import load_dotenv
import glob
import os
from IPython.display import display, Markdown
import chromadb
import json

# Load the variables from .env file and set the API key (or user may manually set the API key)
load_dotenv()  

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = os.getenv('ANTHROPIC_API_KEY')
os.environ["MATHPIX_API_ID"] = os.getenv('MATHPIX_API_KEY')
#openai.api_key = os.getenv('OPENAI_API_KEY')

# Langchain framework
from langchain import hub
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel # for RAG with source
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_experimental.text_splitter import SemanticChunker
from langchain.text_splitter import RecursiveCharacterTextSplitter

## The following loaders are used for options
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import PyPDFium2Loader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders import MathpixPDFLoader
from langchain_community.document_loaders import PDFMinerLoader
from langchain.document_loaders import PyPDFLoader

## 1.2. Initial variable setup

In [3]:
## Initial variable setup
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large")
db_directory = "./data/chroma_semantic"
USE_Anthropic = True

if USE_Anthropic:
    llm = ChatAnthropic(model_name="claude-3-sonnet-20240229", temperature=0)
else:
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) # context window size 16k for GPT 3.5 Turbo

collection_list=[
    "ASOP_life",
    "Bermuda",
    "CFT",
    "PBR",
    "VM21",
    "VM22",
    "Asset",
    "IFRS17"
]
#collection_list = ["PBR"] # for testing

# 2. Load PDF files and convert to a vector DB
## 2.1. Define functions

In [4]:
# Define a function to load and extract text from PDFs in a folder
def get_file_name(source_path):
    return source_path.split('/')[-1]

def load_pdfs_from_folder(folder_path, loader_option):
    # Get a list of PDF files in the specified folder
    pdf_files = glob.glob(f"{folder_path}/*.pdf")
    docs = []
    for pdf_file in pdf_files:
        file_name = get_file_name(pdf_file)
        
        if loader_option == 1:
            # Load the PDF file using the PyPDFLoader
            loader = PyPDFLoader(pdf_file)
        elif loader_option == 2:
            # PyPDFium2Loader is known to be faster than PyPDFLoader
            loader = PyPDFium2Loader(pdf_file)
        elif loader_option == 3:
            # PyMuPDFLoader is known to be general purpose, rich metadata
            loader = PyMuPDFLoader(pdf_file)
        elif loader_option == 4:
            # Allows automated concatenate pages
            loader = PDFMinerLoader(pdf_file, concatenate_pages=True)
        
        loaded_docs = loader.load()
        
        for doc in loaded_docs:
            doc.metadata['source'] = file_name
        
        docs.extend(loaded_docs)
    return docs

def pdf_to_md(folder_path, download_path, loader_option):
    # Get a list of PDF files in the specified folder
    pdf_files = glob.glob(f"{folder_path}/*.pdf")
    for pdf_file in pdf_files:
        file_name = get_file_name(pdf_file)
        base_name = file_name.replace('.pdf', '')
        
        if loader_option == 1:
            # Load the PDF file using the PyPDFLoader
            loader = PyPDFLoader(pdf_file)
        elif loader_option == 2:
            # PyPDFium2Loader is known to be faster than PyPDFLoader
            loader = PyPDFium2Loader(pdf_file)
        elif loader_option == 3:
            # PyMuPDFLoader is known to be general purpose, rich metadata
            loader = PyMuPDFLoader(pdf_file)
        elif loader_option == 4:
            # Allows automated concatenate pages
            loader = PDFMinerLoader(pdf_file, concatenate_pages=True)
        elif loader_option == 5:
            # Use Mathpix OCR to load formula, tables
            # may be slower, but higher quality than all above
            # Require Mathpix API ID - 3 cents per pdf page
            loader = MathpixPDFLoader(pdf_file)
        
        loaded_docs = loader.load()
        
        for i, doc in enumerate(loaded_docs):
            doc.metadata['source'] = file_name
            if loader_option > 3:
                md_file_name = f"{download_path}/{base_name}.md"
            else:
                md_file_name = f"{download_path}/{base_name}{i+1:03d}.md"
            with open(md_file_name, 'w', encoding='utf-8') as md_file:
                md_file.write(doc.page_content)

def load_mds_from_folder(folder_path):
    # Get a list of md files in the specified folder
    md_files = glob.glob(f"{folder_path}/*.md")
    docs = []
    for md_file in md_files:
        file_name = get_file_name(md_file)
        base_name = file_name.replace('.md', '')
        pdf_file_name = f"{base_name}.pdf"
        
        loader = UnstructuredMarkdownLoader(md_file)
        loaded_docs = loader.load()
        
        for doc in loaded_docs:
            doc.metadata['source'] = pdf_file_name
        
        docs.extend(loaded_docs)
    return docs


## 2.2. Convert PDFs to markdown files and then convert to vector database

In [5]:
############################################################################
# Run only to convert pdf to markdown files
############################################################################

collection_list=[
    #"Cayman",
    # "AI_BigData",
    #"ASOP_life",
    "Bermuda",
    # "CFT",
    # "GAAP",
    # "RiskFinance",
    # "PBR",
    # "VM21",
    # "VM22",
    # "Asset",
    # "IFRS17",
]
for collection_name in collection_list:
    # Put new files in the upload subfolder
    folder_path = './data/upload/pdf/'+collection_name
    download_path = './data/upload/md/'+collection_name
    os.makedirs(download_path, exist_ok=True)

    # Use loader option 5 to use Mathpix OCR to load formula, tables
    pdf_to_md(folder_path, download_path, loader_option = 5)

Status: loaded, waiting for processing to complete
Status: split, waiting for processing to complete


In [6]:
############################################################################
# Run to load markdown files to vector database
############################################################################
collection_list=[
    #"Cayman",
    # "AI_BigData",
    #"ASOP_life",
    "Bermuda",
    # "CFT",
    # "GAAP",
    # "RiskFinance",
    # "PBR",
    # "VM21",
    # "VM22",
    # "Asset",
    # "IFRS17",
]

for collection_name in collection_list:
    # Put new files in the upload subfolder
    folder_path = './data/upload/md/'+collection_name

    # Call the function to load and extract text from PDFs in the specified folder
    docs = load_mds_from_folder(folder_path)
    
    # Create a text splitter object with specified parameters
    # text_splitter = RecursiveCharacterTextSplitter(
    #     chunk_size=1000, # 1000 splits a page into roughly 3 chunks
    #     chunk_overlap=200,
    #     length_function=len,)

    # Use semantic chunker to increase meaningfulness of the chunks
    text_splitter = SemanticChunker(embeddings_model)

    # Split the documents into chunks using the text splitter
    splits = text_splitter.split_documents(docs)

    # Create a Chroma vector database from the document splits
    Chroma.from_documents(
        documents=splits, 
        embedding=embeddings_model, 
        persist_directory=db_directory,
        collection_name=collection_name,
    )

## 2.3. (NOT USED) Convert PDFs to vector database directly

In [ ]:
############################################################################
# Run to load pdf files to vector database
############################################################################

for collection_name in collection_list:
    # Put new files in the upload subfolder
    folder_path = './data/upload/pdf/'+collection_name

    # Call the function to load and extract text from PDFs in the specified folder
    docs = load_pdfs_from_folder(folder_path, loader_option = 1)
    
    # Create a text splitter object with specified parameters
    # text_splitter = RecursiveCharacterTextSplitter(
    #     chunk_size=1000, # 1000 splits a page into roughly 3 chunks
    #     chunk_overlap=200,
    #     length_function=len,)

    # Use semantic chunker to increase meaningfulness of the chunks
    text_splitter = SemanticChunker(embeddings_model)

    # Split the documents into chunks using the text splitter
    splits = text_splitter.split_documents(docs)

    # Create a Chroma vector database from the document splits
    Chroma.from_documents(
        documents=splits, 
        embedding=embeddings_model, 
        persist_directory=db_directory,
        collection_name=collection_name,
    )

# 3. For test purposes
## 3.1. Define vector store from vector database 

In [ ]:
## a user may choose different collection name from the list
# ["ASOP_life", "Bermuda", "CFT", "VM21", "VM22", "Asset", "IFRS17"]
collection_name = collection_list[0] 

# Get a Chroma vector database with specified parameters
vectorstore = Chroma(embedding_function=embeddings_model, 
                     persist_directory=db_directory,
                     collection_name=collection_name)

## 3.2. Retrieve from the vector store

In [ ]:
## Retrieve and RAG chain
# Create a retriever using the vector database as the search source
# You may choose a specific document to filter the search
retriever = vectorstore.as_retriever(search_type="mmr", 
                                     search_kwargs={
                                        'k': 6, 
                                        'lambda_mult': 0.5,
                                        # 'filter': {'source': '201611-Guidance-Notes-for-Commercial-Insurers-and-Groups-Statutory-Reporting-Regime-30-Nov-2016.pdf'}
                                        }
                                    ) 
# Use MMR (Maximum Marginal Relevance) to find a set of documents that are both similar to the input query and diverse among themselves
# Increase the number of documents to get, and increase diversity (lambda mult 0.5 being default, 0 being the most diverse, 1 being the least)

# Load the RAG (Retrieval-Augmented Generation) prompt
qa_system_prompt = """You are a helpful assistant to help actuaries with question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
ASOP or asop means Actuarial Standards of Practice. \
CFT means Cash Flow Testing. AAT means Asset Adequacy Testing. \
BMA means Bermuda Monetary Authority. \
SBA means scenario-based approach. BEL means best estimate liabilities.\
After you answer, provide the sources you used to answer the question. \
If you don't know the answer, just say that you don't know. \

{context}"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        ("human", "{question}"),
    ]
)

# Define a function to format the documents with their sources and pages
def format_docs_with_sources(docs):
    formatted_docs = "\n\n".join(doc.page_content for doc in docs)
    #sources_pages = "\n".join(f"{doc.metadata['source']} (Page {doc.metadata['page'] + 1})" for doc in docs)
    sources_pages = "\n".join(f"{doc.metadata['source']})" for doc in docs)
    # Added 1 to the page number assuming 'page' starts at 0 and we want to present it in a user-friendly way

    return f"Documents:\n{formatted_docs}\n\nSources and Pages:\n{sources_pages}"

# Create a RAG chain using the formatted documents as the context
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs_with_sources(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

# Create a parallel chain for retrieving and generating answers
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

## 3.3. Generate Q&A Function

In [ ]:
def generate_output():
    # Prompt the user for a question on ASOP
    usr_input = input("What is your question on ASOP?: ")

    # Invoke the RAG chain with the user input as the question
    output = rag_chain_with_source.invoke(usr_input)

    # Generate the Markdown output with the question, answer, and context
    markdown_output = "### Question\n{}\n\n### Answer\n{}\n\n### Context\n".format(output['question'], output['answer'])

    last_page_content = None  # Variable to store the last page content
    i = 1 # Source indicator

    # Iterate over the context documents to format and include them in the output
    for doc in output['context']:
        current_page_content = doc.page_content.replace('\n', '  \n')  # Get the current page content
        
        # Check if the current content is different from the last one
        if current_page_content != last_page_content:
            #markdown_output += "- **Source {}**: {}, page {}:\n\n{}\n".format(i, doc.metadata['source'], doc.metadata['page'], current_page_content)
            markdown_output += "- **Source {}**: {}:\n\n{}\n".format(i, doc.metadata['source'], current_page_content)
            i = i + 1
        last_page_content = current_page_content  # Update the last page content
    
    # Display the Markdown output
    display(Markdown(markdown_output))

### Example questions related to ASOPs
- explain ASOP No. 14
- How are expenses relfected in cash flow testing based on ASOP No. 22?
- What is catastrophe risk?
- When do I update assumptions?
- What should I do when I do not have credible data to develop non-economic assumptions?

In [ ]:
generate_output()

# 4. References
- https://www.actuarialstandardsboard.org/standards-of-practice/
- https://python.langchain.com/docs/use_cases/question_answering/quickstart
- https://python.langchain.com/docs/use_cases/question_answering/sources
- https://python.langchain.com/docs/integrations/text_embedding/
- https://python.langchain.com/docs/integrations/vectorstores/chroma
- https://docs.gpt4all.io/gpt4all_python_embedding.html#gpt4all.gpt4all.Embed4All
- https://chat.langchain.com/
- https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.chroma.Chroma.html

# 5. Management of the vector database

In [13]:
client = chromadb.PersistentClient(path=db_directory)

In [ ]:
#client.delete_collection(name="VM20") # Delete a collection and all associated files

In [14]:
collection = client.get_collection(name="RiskFinance") 
collection.count()


727

In [6]:
collection.peek(1)

{'ids': ['004c3cdf-7c38-4491-bb2a-fe814ad3daed'],
 'embeddings': [[-0.008267690427601337,
   0.006727959029376507,
   -0.020853325724601746,
   -0.03271930292248726,
   0.005635920912027359,
   0.03497869148850441,
   0.0030920160934329033,
   0.014585613273084164,
   -0.021656664088368416,
   0.028936918824911118,
   0.013983109034597874,
   -0.0008305347873829305,
   -0.03641800582408905,
   0.011581461876630783,
   -0.013313660398125648,
   0.04237610101699829,
   0.0024936962872743607,
   0.012451745569705963,
   -0.023029034957289696,
   -0.013598175719380379,
   -0.007313726935535669,
   -0.053053807467222214,
   -0.017974695190787315,
   0.0005128080956637859,
   0.005351405590772629,
   -0.001486385241150856,
   0.011422468349337578,
   -0.005903700366616249,
   -0.017321983352303505,
   -0.010677706450223923,
   0.01674458384513855,
   0.021790554746985435,
   -0.025489257648587227,
   -0.011129584163427353,
   0.003876526141539216,
   0.002391186775639653,
   0.00498739257454

In [ ]:
#collection.modify(name="PBR") rename

In [15]:
from collections import defaultdict

# Get all the metadatas
metadatas = collection.get()['metadatas']

# Create a dictionary to store distinct source names
distinct_sources = defaultdict(set)

# Iterate through metadatas and add source names to the dictionary
for metadata in metadatas:
    source = metadata.get('source', None)
    if source:
        distinct_sources[source].add(source)

# Get the distinct source names as a list
distinct_source_names = list(distinct_sources.keys())

print(distinct_source_names)

['2016-SOA-ESG_guide.pdf', '2014-SOACIA-ModelValidation_RIsk_Capital_Models.pdf', '2016-SOA-Economic-Capital-report.pdf', '2010-SOA-OperationalRiskPaper.pdf', '2016-AAA-ORSA_Risk_Exposures_PracticeNote.pdf', '2019-AAA-ModelRiskManagementPracticeNote.pdf', '2023-SOA-reg-capital-comparison.pdf', '2018-AAA-Life_Reins_Reserve_Credit_Practice_Note.pdf', '2016-SOA-nested-stochastic-modeling-report.pdf', '2022-SOA-RW_vs_RN_scenarios.pdf', '2011-SOA-Model_efficiency_study.pdf', '2024-AAA-liquidity-risk_practice_note.pdf', '2024-AAA-issue-brief-group-capital.pdf', '2023-SOA-rating-agency-perspectives.pdf', '2023-SOA-interest-rate-model-calibration.pdf', '2020-AAA-Tax_Cuts_Jobs_ACT_White_Paper.pdf', '2023-AAA-issue-brief-dividend-limitation.pdf', '2009-SOA-EmbeddedValue_practice_theory.pdf']


In [19]:
collection.get(
    where={"source": "2023-SOA-interest-rate-model-calibration.pdf"}
)

{'ids': ['3162223f-400f-4815-8c90-e270ad806f16'],
 'embeddings': None,
 'metadatas': [{'source': '2023-SOA-interest-rate-model-calibration.pdf'}],
 'documents': ['{"error":"Internal error","error_info":{"id":"sys_exception","message":"Internal error"}}'],
 'uris': None,
 'data': None}

In [20]:
collection.delete(
    #ids=["id1", "id2", "id3",...],
    where={"source": '2023-SOA-interest-rate-model-calibration.pdf'}
)

OperationalError: attempt to write a readonly database

# Rename process

In [71]:
# Rename process
folder_name = "VM22"
old_file_name = "VM-22 Subgroup Draft July 2023 Clean.pdf"
new_file_name = "202307-NAIC-VM-22 Subgroup Draft.pdf"

# Chroma DB update
collection = client.get_collection(name=folder_name)
results = collection.get(where={"source": old_file_name})
ids_to_update = results['ids']
print(ids_to_update)
new_metadatas = [{"source": new_file_name} for _ in ids_to_update]
collection.update(ids=ids_to_update, metadatas=new_metadatas)


['0155bc1d-6072-42dd-9eeb-f9fa8963e64a', '01e9a1cd-8fd3-4862-8da9-fa5c289478e2', '027ea317-f4cf-48cd-91fc-6fe941f13c1b', '02a1588f-e491-41df-8092-0e204ff7b6bd', '04eac448-61a0-4a52-97ec-c69456610fcc', '094efb4a-5ad4-43fa-b567-870f57ae6483', '0fa8c66a-2009-403b-8d0e-6a497177aefa', '16f97770-a6b1-47b5-994f-01401ac1900a', '18bdc3dc-58cc-4888-bfb8-7db46ab0a6cc', '1ac0c827-d089-48b4-8ee9-427239ee000b', '1d51fd56-722a-4dee-bc70-f768663e345f', '210e0a7d-8421-4461-9232-542d48872a23', '21dc59bd-8735-4f43-a69d-061a0799b092', '22c0eb0f-6ec5-4a54-b47c-3f3f58a7befd', '247ade0b-393b-4125-8ca7-ab66bcf1aa55', '24b15100-ee14-403a-bccd-912797e43dc4', '28eba040-ec18-440b-becf-5f189c9040d9', '293b4d74-6dc5-47e8-a1a4-8ea10d1a0f40', '297163cf-690d-4340-aa70-a38d69f19fcd', '2d080af5-9e66-423b-8661-1a59053f2de7', '323b78dd-bc3e-49cd-a1f4-993b49bbd2ae', '32e39474-f1d0-48e7-b659-b99b2fdc8f81', '3721f475-26dc-4b34-b8be-f04f2eda5696', '3861152c-fb17-4dc7-833c-3bf91f6c1aec', '42d7ca09-3fc5-4391-981d-6c9c19c337e6',

In [72]:

# json file summary update
with open('summary.json', 'r') as f:
    data = json.load(f)
# Update the existing data with the new key-value pair
data[new_file_name] = data.pop(old_file_name)

with open('summary.json', 'w') as f:
    json.dump(data, f, indent=4)


In [73]:

# md and pdf file update
# Construct the paths to the folders
md_folder = os.path.join("data/md", folder_name)
pdf_folder = os.path.join("data/pdf", folder_name)

# Construct the old and new file paths
old_file_name_md = old_file_name.replace(".pdf", ".md")
new_file_name_md = new_file_name.replace(".pdf", ".md")
old_md_path = os.path.join(md_folder, old_file_name_md)
new_md_path = os.path.join(md_folder, new_file_name_md)

# Check if the old MD file exists before renaming
if os.path.exists(old_md_path):
    # Rename the MD file
    os.rename(old_md_path, new_md_path)
else:
    print(f"Skipping MD file rename: {old_md_path} does not exist.")

# Construct the old and new file paths
old_pdf_path = os.path.join(pdf_folder, old_file_name)
new_pdf_path = os.path.join(pdf_folder, new_file_name)

# Rename the file
os.rename(old_pdf_path, new_pdf_path)

Skipping MD file rename: data/md/VM22/VM-22 Subgroup Draft July 2023 Clean.md does not exist.
